I need to count all the frequencies of all the triplets in each of the video sequence.

In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Counting for the detection task

1.  I need to get the AP for the detection results. 
2.  I need to check how many ground truth were available and how this is related to NaNs
3. I need to check how many predictions were available and how this is related to 

In [17]:
results_triplet_detection = [np.nan, np.nan, np.nan, np.nan, 0.0, np.nan, 0.0, 0.0, np.nan, np.nan, np.nan, 0.0, 0.3323326024624726, np.nan, 0.0, np.nan, np.nan, 0.5142305155955997, 0.0,
 0.4156828953037484, 0.2181818181818182, 0.0, 0.0, np.nan, 0.0, 0.0, np.nan, 0.14545454545454548, 0.0, 0.41837732160312807, np.nan, 
 np.nan, 0.0, np.nan, 0.0, np.nan, 0.07575757575757576, np.nan, np.nan, 0.0, 0.0, np.nan, np.nan, 0.0, np.nan, 0.0, np.nan, np.nan, np.nan, np.nan, np.nan, 0.0, 0.0, 
 np.nan, np.nan, np.nan, np.nan, 0.0, 0.2380308367112467, 0.0, 0.4610338956727119, 0.09090909090909091, 0.0, 0.0, 0.0, np.nan, np.nan, np.nan, 0.0, 
 0.153170359052712, 0.0, np.nan, np.nan, np.nan, 0.0, 0.0, np.nan, np.nan, 0.0, 0.5622358288858232, np.nan, 0.0, 0.4610540358595872, np.nan, np.nan, 
 np.nan, np.nan, np.nan, 0.0, 0.0, 0.0, 0.0, 0.0, np.nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3636363636363637]
results_triplet_detection = np.array(results_triplet_detection)

In [18]:
# Count NaNs
nan_count = np.sum(np.isnan(results_triplet_detection))
print("Number of NaNs:", nan_count)
print(np.nanmean(results_triplet_detection))

Number of NaNs: 46
0.08240903120530413


In [19]:
results_triplet_detection == 0.0

array([False, False, False, False,  True, False,  True,  True, False,
       False, False,  True, False, False,  True, False, False, False,
        True, False, False,  True,  True, False,  True,  True, False,
       False,  True, False, False, False,  True, False,  True, False,
       False, False, False,  True,  True, False, False,  True, False,
        True, False, False, False, False, False,  True,  True, False,
       False, False, False,  True, False,  True, False, False,  True,
        True,  True, False, False, False,  True, False,  True, False,
       False, False,  True,  True, False, False,  True, False, False,
        True, False, False, False, False, False, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False])

In [20]:
zero_count = np.sum(results_triplet_detection == 0.0)
print("Number of zeros:", zero_count)

Number of zeros: 40


In [22]:
non_zero_count = 100 - np.sum(np.isnan(results_triplet_detection)) - np.sum(results_triplet_detection == 0.0)
print("Number of values predicted:", non_zero_count)

Number of values predicted: 14


This is daming. That means, there are only 14 predictions which have some value, while the other 40 predictions, I just got 0. 

For this to be correct, the 14 have to have ground-truth which I got correctly. 

The 40 has to be a mix of 0 ground truths and some predictions or 0 predictions and some ground-truths 

In [23]:
nan_indices = np.where(np.isnan(results_triplet_detection))[0]  # [0] extracts the indices from the tuple

print("Indices of NaNs:", nan_indices)

Indices of NaNs: [ 0  1  2  3  5  8  9 10 13 15 16 23 26 30 31 33 35 37 38 41 42 44 46 47
 48 49 50 53 54 55 56 65 66 67 71 72 73 76 77 80 83 84 85 86 87 93]


In [29]:
zero_indices = np.where(results_triplet_detection == 0.0)[0]  # [0] extracts the indices from the tuple

print("Indices of zeros:", zero_indices)

Indices of zeros: [ 4  6  7 11 14 18 21 22 24 25 28 32 34 39 40 43 45 51 52 57 59 62 63 64
 68 70 74 75 78 81 88 89 90 91 92 94 95 96 97 98]


In [28]:
non_zero_indices = np.where( (results_triplet_detection > 0.0) )[0]  # [0] extracts the indices from the tuple

print("Indices of non_zeros:", non_zero_indices)

Indices of non_zeros: [12 17 19 20 27 29 36 58 60 61 69 79 82 99]


In [30]:
from utils.general.dataset_variables import TripletSegmentationVariables
from os.path import join

In [40]:
import json
import os
from collections import Counter
from typing import Dict, List, Tuple

def count_triplets_in_detections(test_ann_dir: str, triplet_mapping: Dict[str, str]) -> Tuple[Dict[str, int], List[str]]:
    """
    Counts the frequency of triplets in detected LabelMe-style annotations.

    Args:
        test_ann_dir (str): Directory containing LabelMe JSON annotation files.
        triplet_mapping (Dict[str, str]): Dictionary mapping triplet names to a standard format.

    Returns:
        Tuple[Dict[str, int], List[str]]:
            - Dictionary with triplet names as keys and their counts as values.
            - List of triplets in triplet_mapping but missing from the dataset.
    """
    triplet_counter = Counter()

    # Iterate through all JSON files in the test directory
    for file_name in os.listdir(test_ann_dir):
        if file_name.endswith(".json"):
            file_path = os.path.join(test_ann_dir, file_name)
            
            with open(file_path, "r") as f:
                data = json.load(f)

            # Iterate through shapes in the LabelMe JSON file
            triplet_name_with_group_id_list = []
            for annotation in data.get("shapes", []):
                label = annotation.get("label", "").strip().lower()
                verb = annotation.get("verb", "").strip().lower()
                target = annotation.get("target", "").strip().lower()
                group_id = annotation.get("group_id", None)  # Group ID helps determine instance uniqueness
                
                # Construct triplet name
                triplet_name = f"{label},{verb},{target}"
                
                triplet_name_with_group_id = f"{label},{verb},{target},{group_id}"

                
                # Check if the triplet exists in the mapping
                if triplet_name in triplet_mapping.values(): 
                    if triplet_name_with_group_id not in triplet_name_with_group_id_list:
                        triplet_counter[triplet_name] += 1 # add to the counter
                        triplet_name_with_group_id_list.append(triplet_name_with_group_id)  # ensure multiple contours with same group_id not aded
                    else: 
                        print(f"{file_name} - Same group id: {triplet_name_with_group_id} already exists")
                else:
                    print(f"{file_name} - Warning: Triplet {triplet_name} in {file_name} not found in triplet_mapping.")

    # Identify missing triplets
    present_triplets = set(triplet_counter.keys())
    all_triplets = set(triplet_mapping.values())
    triplets_not_in_test_dataset = sorted(all_triplets - present_triplets)

    return dict(triplet_counter), triplets_not_in_test_dataset



In [41]:
                
# Example Usage:
test_ann_dir = "../data/triplet_segmentation_dataset_v2_second_stage/test/ann_dir"  # Replace with actual directory
triplet_mapping = TripletSegmentationVariables.categories['triplet']

triplet_counts, triplets_not_in_test_dataset = count_triplets_in_detections(test_ann_dir, triplet_mapping)


t50_VID14_000190.json - Same group id: bipolar,coagulate,gallbladder,1 already exists
t50_VID14_000265.json - Same group id: grasper,retract,gallbladder,1 already exists
t50_VID14_000502.json - Same group id: hook,dissect,cystic_duct,1 already exists
t50_VID14_000566.json - Same group id: hook,dissect,gallbladder,1 already exists
t50_VID14_000572.json - Same group id: hook,dissect,gallbladder,1 already exists
t50_VID14_000582.json - Same group id: hook,retract,gallbladder,1 already exists
t50_VID14_000606.json - Same group id: clipper,clip,cystic_artery,1 already exists
t50_VID14_000607.json - Same group id: clipper,null_verb,null_target,1 already exists
t50_VID14_000610.json - Same group id: clipper,clip,cystic_artery,1 already exists
t50_VID14_000707.json - Same group id: grasper,retract,gallbladder,1 already exists
t50_VID14_000838.json - Same group id: grasper,retract,gallbladder,1 already exists
t50_VID14_000839.json - Same group id: grasper,retract,gallbladder,1 already exists
t5

In [42]:
print("Triplet Counts:", triplet_counts)
print("triplets_not_in_test_dataset:", triplets_not_in_test_dataset)

Triplet Counts: {'grasper,retract,gallbladder': 1528, 'hook,dissect,omentum': 120, 'grasper,retract,omentum': 76, 'grasper,retract,gut': 28, 'hook,coagulate,gallbladder': 11, 'bipolar,coagulate,gallbladder': 12, 'bipolar,dissect,gallbladder': 37, 'hook,dissect,gallbladder': 1367, 'hook,null_verb,null_target': 55, 'hook,dissect,peritoneum': 18, 'grasper,retract,peritoneum': 25, 'hook,dissect,cystic_plate': 147, 'hook,retract,gallbladder': 20, 'hook,dissect,cystic_artery': 80, 'hook,dissect,cystic_duct': 187, 'clipper,clip,cystic_duct': 49, 'clipper,clip,cystic_artery': 32, 'clipper,null_verb,null_target': 17, 'scissors,cut,cystic_duct': 24, 'scissors,cut,cystic_artery': 14, 'bipolar,coagulate,liver': 148, 'irrigator,aspirate,fluid': 361, 'grasper,null_verb,null_target': 127, 'irrigator,irrigate,abdominal_wall_cavity': 26, 'irrigator,null_verb,null_target': 55, 'hook,coagulate,liver': 11, 'grasper,retract,liver': 829, 'hook,retract,liver': 13, 'bipolar,coagulate,cystic_plate': 139, 'irri

In [43]:
len(triplets_not_in_test_dataset)

51

In [ ]:
all_triplets_set = set(triplet_mapping.values())
triplets_not_in_test_dataset = set(triplets_not_in_test_dataset)
triplets_in_test_dataset = set(triplet_counts.keys())

In [65]:
nan_triplet_names = set([triplet_mapping[str(int(id)+1)] for id in nan_indices])
non_zero_indices_names = set([triplet_mapping[str(int(id)+1)] for id in non_zero_indices])
zero_indices_names  = set([triplet_mapping[str(int(id)+1)] for id in zero_indices])

In [57]:
all_nans_that_do_not_have_gt = triplets_not_in_test_dataset & nan_triplet_names

In [60]:
all_nans_that_do_not_have_gt == nan_triplet_names

True

In [62]:
print(len(all_nans_that_do_not_have_gt))
print(len(triplets_not_in_test_dataset))
print(len(nan_triplet_names))

46
51
46


ohh okay. For all NaNs, they are things that do not have ground-truth. So for me to have a NaN, I must have no groundtruth. 

But there are 5 things which there is no ground_truth, but they are not nans.  

I suspect that they are zeros. The ones we cannot find are zeros. 

In [64]:
triplets_which_have_no_gt_but_are_not_nans =  triplets_not_in_test_dataset - nan_triplet_names
triplets_which_have_no_gt_but_are_not_nans

{'bipolar,dissect,adhesion',
 'bipolar,dissect,cystic_duct',
 'grasper,grasp,peritoneum',
 'grasper,retract,cystic_duct',
 'scissors,dissect,gallbladder'}

if I am right, these are the triplets which are not in test, but were predicted anyway, they should give zeros

In [66]:
zero_scores_which_do_not_exist_in_test = zero_indices_names & triplets_not_in_test_dataset
zero_scores_which_do_not_exist_in_test

{'bipolar,dissect,adhesion',
 'bipolar,dissect,cystic_duct',
 'grasper,grasp,peritoneum',
 'grasper,retract,cystic_duct',
 'scissors,dissect,gallbladder'}

So I am correct, and the metric is correct. 

It also follows that the 0.0 which are not in zero_scores_which_do_not_exist_in_test are things which we predicted wrongly entirely. 

While the other ones, are things we at least got correctly done.